<a href="https://colab.research.google.com/github/SlowPoke228/Lessons/blob/main/L3(Linear).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
drive.mount('/content/drive')
# url db: https://www.kaggle.com/christianlillelund/csgo-round-winner-classification
# url lesson: http://slides.com/romvano/lab3-6#/6


names = ['time_left','ct_score','t_score','map','bomb_planted','ct_health','t_health','ct_armor','t_armor','ct_money','t_money','ct_helmets',
        't_helmets','ct_defuse_kits','ct_players_alive','t_players_alive','ct_weapon_ak47','t_weapon_ak47','ct_weapon_aug','t_weapon_aug','ct_weapon_awp'
        ,'t_weapon_awp','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','t_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_m4a4','t_weapon_m4a4','ct_weapon_mac10','t_weapon_mac10','ct_weapon_mag7'
         ,'t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','ct_weapon_mp9','t_weapon_mp9','ct_weapon_negev','t_weapon_negev','ct_weapon_nova',
        't_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver','ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20',
       't_weapon_scar20','ct_weapon_sg553','t_weapon_sg553','ct_weapon_ssg08','t_weapon_ssg08','ct_weapon_ump45','t_weapon_ump45','ct_weapon_xm1014',
       't_weapon_xm1014','ct_weapon_deagle','t_weapon_deagle','ct_weapon_fiveseven','t_weapon_fiveseven','ct_weapon_usps','t_weapon_usps','ct_weapon_p250','t_weapon_p250',
       'ct_weapon_p2000','t_weapon_p2000','ct_weapon_tec9','t_weapon_tec9','ct_grenade_hegrenade','t_grenade_hegrenade','ct_grenade_flashbang','t_grenade_flashbang',
       'ct_grenade_smokegrenade','t_grenade_smokegrenade','ct_grenade_incendiarygrenade','t_grenade_incendiarygrenade','ct_grenade_molotovgrenade','t_grenade_molotovgrenade',
       'ct_grenade_decoygrenade','t_grenade_decoygrenade','round_winner'
]

# Подключаем бд, удаляем первую стоку и указываем тайтлы
dataset = pd.read_csv("/content/drive/MyDrive/csgo_round_snapshots.csv", names=names, skiprows=1)
#удаляем данные с слишком высокой и слишком низкой корреляцией
dataset = dataset.drop(['map','bomb_planted','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_mac10','ct_weapon_mag7','t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','t_weapon_mp9',
        'ct_weapon_negev','t_weapon_negev','ct_weapon_nova','ct_weapon_xm1014','t_weapon_xm1014','t_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver',
        'ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20','t_weapon_scar20','ct_grenade_decoygrenade','t_grenade_decoygrenade','t_grenade_incendiarygrenade','ct_weapon_p2000','t_weapon_p2000',
        'ct_weapon_tec9','t_weapon_tec9','t_weapon_aug','ct_players_alive','t_players_alive','t_weapon_m4a4','t_weapon_ssg08','ct_grenade_molotovgrenade','ct_weapon_fiveseven','t_weapon_fiveseven'], 
        axis='columns')
dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_glock,ct_weapon_m4a4,t_weapon_mac10,ct_weapon_mp9,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_deagle,t_weapon_deagle,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_molotovgrenade,round_winner
0,175.00,0.0,0.0,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
2,96.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,CT
3,76.03,0.0,0.0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,T
122406,174.93,11.0,15.0,500.0,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,4.0,4.0,3.0,5.0,1.0,5.0,T
122408,94.93,11.0,15.0,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0,0.0,3.0,0.0,4.0,T


In [ ]:

# Создаем массивы с атрибутами и метками
X = dataset.iloc[:, :-1].values    
y = dataset.iloc[:, len(dataset.columns)-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=42,stratify=y)

Сравнение L1 и L2 регулизаций, подбор параметров, получение весов признаков

In [ ]:
C_list = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]

for m in ['l1','l2']:
  max_score=0
  max_score_c=0
  print('Тип регуляризации:',m)
  for i in range(0, len(C_list)):
    pipe = Pipeline([('scl', StandardScaler()),('reg', LogisticRegression(C=C_list[i], random_state=42,penalty=m,solver='saga'))])  
    pipe.fit(X_train, y_train) 
    sc=pipe.score(X_test, y_test)
    print('C=',C_list[i],'score:',sc)
    print(pipe.named_steps['reg'].coef_,'\n')
    if max_score<sc:
      max_score=sc
      max_score_c=C_list[i]
  print('Максимальная точность:',max_score,'при С =',max_score_c) 

Тип регуляризации: l1
C= 1e-05 score: 0.5098439669961604
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

C= 0.0001 score: 0.7269830896168614
[[ 0.          0.          0.          0.          0.         -0.40070309
   0.24302801  0.          0.         -0.03804325  0.12782565 -0.00300236
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]] 

C= 0.001 score: 0.73915529776979
[[ 0.          0.          0.         -0.66217766  0.58763531 -0.79403396
   0.6618426  -0.10384248  0.08589569 -0.10349235  0.36006063 -0.014712
  -0.02214492  0.01541835 -0.03085997 -0.09884904  0.02443881  0.
  -0.10084372  0.          0.         -0.04009206  0.03262832  0.
  

Отберем признаки и обучим алгоритм методом ближайших соседий

In [ ]:
dataset = dataset.drop(['time_left','ct_score','t_score','ct_health','ct_defuse_kits','t_weapon_glock','t_weapon_mac10','ct_weapon_mp9','ct_weapon_ssg08','ct_weapon_ump45',	't_weapon_ump45',	'ct_weapon_deagle'	,'t_weapon_deagle',	'ct_weapon_usps',	't_weapon_usps',	'ct_weapon_p250',	't_weapon_p250',	'ct_grenade_hegrenade',	't_grenade_hegrenade',		't_grenade_flashbang',	'ct_grenade_smokegrenade',		'ct_grenade_incendiarygrenade'], 
        axis='columns')
# Создаем массивы с атрибутами и метками
X = dataset.iloc[:, :-1].values    
y = dataset.iloc[:, len(dataset.columns)-1].values
dataset

,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,ct_weapon_awp,t_weapon_awp,ct_weapon_m4a4,ct_weapon_sg553,t_weapon_sg553,ct_grenade_flashbang,t_grenade_smokegrenade,t_grenade_molotovgrenade,round_winner
0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,500.0,400.0,300.0,600.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,CT
2,400.0,294.0,200.0,750.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,CT
3,400.0,294.0,200.0,750.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,T
122406,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,T
122407,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,4.0,5.0,5.0,T
122408,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0,4.0,T


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
# разделяем массивы на тренеровочные и тестовые 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=42)

# Перебор значений К 
error = []
for i in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean(pred_i != y_test))
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# пересчет с оптимальным значением соседий
classifier = KNeighborsClassifier(n_neighbors=error.index(min(error))+1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
# вывод метрик
cf=confusion_matrix(y_test, y_pred)
print('При К равным: ', error.index(min(error))+1)
print(cf)
print(classification_report(y_test, y_pred))

При К равным:  1
[[4867 1133]
 [1096 5145]]
              precision    recall  f1-score   support

          CT       0.82      0.81      0.81      6000
           T       0.82      0.82      0.82      6241

    accuracy                           0.82     12241
   macro avg       0.82      0.82      0.82     12241
weighted avg       0.82      0.82      0.82     12241

